In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [2]:
import tensorflow as tf

2025-12-02 23:29:43.207288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-02 23:29:43.276420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-02 23:29:43.293128: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-02 23:29:43.392009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-02 23:29:45.448581: W tensorflow/compiler/tf2

In [3]:
from typing import Any

In [4]:
from mobilenetv2ssd.core.config import load_config

In [5]:
from mobilenetv2ssd.models.ssd.model import SSD

## Config Files To Test Orchestration¶

In [6]:
main_cfg_path = "configs/train/default.yaml"
model_cfg_path = "configs/model/mobilenetv2_ssd_voc.yaml"
data_cfg_path = "configs/data/voc_224.yaml"
eval_cfg_path = "configs/eval/default.yaml"

In [7]:
config = load_config(main_cfg_path,model_cfg_path,data_cfg_path,eval_cfg_path)

## Helper Functions For Orchestration

In [8]:
def _extract_information_from_train_config(config : dict[str, Any]):
    
    model_config = config['model']
    
    # Getting the specs for the model 
    input_shape = model_config.get('input_size',[300,300,3])
    alpha = model_config.get('width_mult',1.0)
    
    backbone_type = model_config.get('backbone','mobilenetv2')
    ssd_name = model_config.get('name',"mobilenetv2-ssd")
    
    num_classes = model_config['num_classes']
    feature_map_config = model_config['ssd']['feature_maps']
    
    backbone_features = feature_map_config.get('backbone_features', ["C3", "C4", "C5"])
    
    extra_base = feature_map_config.get('extra_base', None)
    
    extra_levels = feature_map_config.get('extra_levels', [{'name': 'P6', 'out_channels': 256, 'stride': 2, 'kernel_size': 3},{'name': 'P7', 'out_channels': 256, 'stride': 2, 'kernel_size': 3},{'name': 'P8', 'out_channels': 128, 'stride': 2, 'kernel_size': 3}])
    
    head_config = model_config.get("heads",{})
    localization_config = head_config.get('localization',{})
    classification_config = head_config.get('classification',{})

    return input_shape, alpha, backbone_type, ssd_name, num_classes, backbone_features, localization_config, classification_config,extra_levels, extra_base

In [12]:
_extract_information_from_train_config(config)

([224, 224, 3],
 1.0,
 'mobilenetv2',
 'mobilenetv2-ssd',
 21,
 ['C3', 'C4', 'C5'],
 {'norm_cfg': 'batch_norm',
  'head_type': 'conv3x3',
  'intermediate_channels': 256,
  'squeeze_ratio': 1.0},
 {'norm_cfg': 'batch_norm',
  'head_type': 'conv3x3',
  'intermediate_channels': 256,
  'squeeze_ratio': 1.0,
  'use_sigmoid_cls': False},
 [{'name': 'P6', 'out_channels': 256, 'stride': 2, 'kernel_size': 3},
  {'name': 'P7', 'out_channels': 256, 'stride': 2, 'kernel_size': 3},
  {'name': 'P8', 'out_channels': 128, 'stride': 2, 'kernel_size': 3}],
 'C5')

## Factory Function

In [13]:
def build_ssd_model(config: dict[str,Any], anchors_per_layer: list[int]):
    # Read from config
    input_shape, alpha, backbone_type, ssd_name, num_classes, backbone_features, localization_config, classification_config,extra_levels, extra_base = _extract_information_from_train_config(config)
    
    # Creating the SSD model
    ssd = SSD(backbone_type = backbone_type, name = ssd_name, feature_maps = backbone_features, number_of_classes = num_classes, number_of_anchors_per_layer = anchors_per_layer, input_shape = tuple(input_shape), loc_head_configuration =localization_config , cls_head_configuration = classification_config, extra_levels = extra_levels, extra_base = extra_base, alpha = alpha)

    dummy_shape = [1] + input_shape

    dummy_image = tf.random.uniform(dummy_shape, dtype=tf.float32)

    ssd(dummy_image)

    return ssd

In [14]:
build_ssd_model(config,[ 6, 10, 10,  6,  6,  6])

[build_backbone] Loading existing weights from: /mnt/d/dev/MobileNetV2-SSD/src/mobilenetv2ssd/models/mobilenet_v2/weights/mobilenetv2_imagenet_notop_224x224.weights.h5


<SSD name=mobilenetv2-ssd, built=True>